## Import Libraries

In [1]:
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 92.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [2]:
!pip install transformers

In [3]:
!pip install -U sentence-transformers

In [4]:
!pip install xgboost

In [6]:
#Imports required packages
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from lightgbm import LGBMRegressor
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertModel, DistilBertConfig, RobertaConfig, AutoTokenizer, AutoModel
from adapters import AdapterConfig, DistilBertAdapterModel, RobertaAdapterModel
import torch.nn as nn
import re
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## Dataset Download

In [7]:
!wget https://raw.githubusercontent.com/nlbse2024/issue-report-classification/main/data/issues_train.csv
!wget https://raw.githubusercontent.com/nlbse2024/issue-report-classification/main/data/issues_test.csv

--2025-01-13 02:12:55--  https://raw.githubusercontent.com/nlbse2024/issue-report-classification/main/data/issues_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3680400 (3.5M) [text/plain]
Saving to: ‘issues_train.csv’

issues_train.csv    100%[===================>]   3.51M  --.-KB/s    in 0.02s   

2025-01-13 02:12:55 (224 MB/s) - ‘issues_train.csv’ saved [3680400/3680400]

--2025-01-13 02:12:55--  https://raw.githubusercontent.com/nlbse2024/issue-report-classification/main/data/issues_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting res

In [8]:
# loading training data
traindf = pd.read_csv('issues_train.csv')

# loading testing data
testdf = pd.read_csv('issues_test.csv')

In [9]:
traindf

,repo,created_at,label,title,body
0,facebook/react,2023-08-26 06:33:37,bug,"[DevTools Bug] Cannot add node ""1"" because a n...",### Website or app\n\nPrivate repo cannot give...
1,facebook/react,2023-07-28 05:16:12,bug,[DevTools Bug]: Devtools extension build faili...,### Website or app\n\nN/A\n\n### Repro steps\n...
2,facebook/react,2023-07-13 21:58:31,bug,[DevTools Bug]: Deprecated __REACT_DEVTOOLS_GL...,### Website or app\n\nhttps://github.com/open-...
3,facebook/react,2023-06-14 02:31:20,bug,"[DevTools Bug] Cannot remove node ""0"" because ...",### Website or app\n\nlocal\n\n### Repro steps...
4,facebook/react,2023-06-03 11:29:44,bug,"[DevTools Bug] Cannot remove node ""103"" becaus...",### Website or app\n\nlocalhost\n\n### Repro s...
...,...,...,...,...,...
1495,opencv/opencv,2022-01-24 10:48:13,feature,core: FP denormals support,relates #21046\r\n\r\n- support x86 SSE FTZ+DA...
1496,opencv/opencv,2022-01-20 12:40:55,feature,feature: submodule or a class scope for export...,All classes are registered in the scope that c...
1497,opencv/opencv,2022-01-15 02:39:22,feature,Reading BigTiff images,**Merge with extra: https://github.com/opencv/...
1498,opencv/opencv,2022-01-14 15:37:53,feature,Add general broadcasting layer,Performance details(broadcasting 1x1 to 16x204...


In [10]:
testdf

,repo,created_at,label,title,body
0,facebook/react,2023-08-02 02:26:00,bug,Bug: [18.3.0-canary] renderToString hoists som...,<!--\r\n Please provide a clear and concise d...
1,facebook/react,2023-07-17 22:43:05,bug,[DevTools Bug]: Chrome extension gets disconne...,### Website or app\r\n\r\nhttps://react.dev/\r...
2,facebook/react,2023-07-13 19:01:47,bug,[DevTools Bug]: Deprecated __REACT_DEVTOOLS_GL...,### Website or app\n\nN/A\n\n### Repro steps\n...
3,facebook/react,2023-06-07 17:26:43,bug,[DevTools Bug]: React devtools stuck at Loadin...,### Website or app\n\ncorporate project (priva...
4,facebook/react,2023-05-31 15:17:41,bug,Bug: Radio button onChange not called in curre...,<!--\r\n Please provide a clear and concise d...
...,...,...,...,...,...
1495,opencv/opencv,2022-01-22 11:52:21,feature,Task: GCC 12 support,Support compilation with GCC 12 and fix tests\...
1496,opencv/opencv,2022-01-16 19:27:55,feature,AudioIO: add dnn speech recognition sample on C++,### Pull Request Readiness Checklist\r\n\r\nSe...
1497,opencv/opencv,2022-01-14 22:05:58,feature,Use modern OpenVINO package interface,"* new cmake options: `WITH_OPENVINO`, `OPENCV_..."
1498,opencv/opencv,2022-01-12 09:14:41,feature,TiffEncoder write support more depth type,**Merge with extra**: https://github.com/openc...


## Dataset Preprocessing

In [11]:
traindf.drop(['repo','created_at'],axis=1,inplace=True)
testdf.drop(['repo','created_at'],axis=1,inplace=True)

In [12]:
# checking the shape of the data
traindf.shape,testdf.shape

((1500, 3), (1500, 3))

In [13]:
print(traindf.isnull().sum())
print(testdf.isnull().sum())

label    0
title    0
body     0
dtype: int64
label    0
title    0
body     2
dtype: int64


In [14]:
traindf.dropna(inplace = True)
testdf.dropna(inplace = True)

In [15]:
le = LabelEncoder()
traindf['label'] = le.fit_transform(traindf['label'])
testdf['label'] = le.transform(testdf['label'])

In [16]:
def preprocess(issues):
    processed_issues = []

    for issue in issues:

        if not isinstance(issue, str):
            issue = str(issue) if issue is not None else ''

        # Remove strings between triple quotes
        issue = re.sub(r'```.*?```', ' ', issue, flags=re.DOTALL)

        # Remove new lines
        issue = re.sub(r'\n', ' ', issue)

        # Remove links
        issue = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', issue)

        # Remove digits
        issue = re.sub(r'\d+', ' ', issue)

        # Remove special characters except the question marks
        issue = re.sub(r'[^a-zA-Z0-9?\s]', ' ', issue)
        issue = re.sub(r'\s+', ' ', issue)

        processed_issues.append(issue)

    return processed_issues

In [17]:
traindf['title'] = preprocess(traindf['title'])
traindf['body'] = preprocess(traindf['body'])

testdf['title'] = preprocess(testdf['title'])
testdf['body'] = preprocess(testdf['body'])

# Generating Embeddings

### Sentence Transformer

In [18]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the Transformer
model1 = SentenceTransformer("all-mpnet-base-v2")

# Batch size
batch_size = 16

# Function to create a DataLoader that splits the text data into batches
def create_dataloader(text_data, batch_size):
    return DataLoader(text_data, batch_size=batch_size, shuffle=False)

# Function to process each batch
def process_batches(dataloader):
    all_cls_embeddings = []

    for batch in dataloader:
        # Get hidden states without gradient calculation
        with torch.no_grad():
            hidden_states = model1.encode(batch)

        # Extract [CLS] token's hidden state for each sentence in the batch
        cls_embeddings = hidden_states[:, :]  # Shape: [batch_size, emb_dim]
        all_cls_embeddings.append(cls_embeddings)

    # Concatenate all the embeddings for all batches
    all_cls_embeddings = [torch.from_numpy(embedding) if isinstance(embedding, np.ndarray) else embedding for embedding in all_cls_embeddings]
    all_cls_embeddings = torch.cat(all_cls_embeddings, dim=0)

    return all_cls_embeddings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Roberta-base

In [19]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer and model for RoBERTa
tokenizer_roberta = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model_roberta = AutoModel.from_pretrained("FacebookAI/roberta-base").to(device)

# Set batch size
batch_size = 16  # You can adjust this based on your memory constraints

# Create DataLoader for batching the input text
def create_dataloader(text_data, batch_size):
    return DataLoader(text_data, batch_size=batch_size, shuffle=False)

# Function to process each batch and extract [CLS] token embeddings
def process_batches_roberta(dataloader):
    all_cls_embeddings = []

    for batch in dataloader:
        # Tokenize the batch
        tokenized_batch = tokenizer_roberta(list(batch), padding=True, truncation=True, return_tensors="pt")

        # Move tokenized data to the appropriate device (GPU/CPU)
        tokenized_batch = {k: v.to(device) for k, v in tokenized_batch.items()}

        # Forward pass to get the hidden states without gradient calculation
        with torch.no_grad():
            hidden_states = model_roberta(**tokenized_batch)

        # Extract [CLS] token's hidden state for each sentence in the batch
        cls_embeddings = hidden_states.last_hidden_state[:, 0, :]  # Shape: [batch_size, emb_dim]
        all_cls_embeddings.append(cls_embeddings)

    # Concatenate all the [CLS] embeddings for all batches
    all_cls_embeddings = torch.cat(all_cls_embeddings, dim=0)

    return all_cls_embeddings

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
##Sentence Transformer
# Body
train_dataloader = create_dataloader(traindf["body"].values.tolist(), batch_size)
test_dataloader = create_dataloader(testdf["body"].values.tolist(), batch_size)
cls_train_body_sent = process_batches(train_dataloader)
cls_test_body_sent = process_batches(test_dataloader)

# Title
train_dataloader_title = create_dataloader(traindf["title"].values.tolist(), batch_size)
test_dataloader_title = create_dataloader(testdf["title"].values.tolist(), batch_size)
cls_train_title_sent = process_batches(train_dataloader_title)
cls_test_title_sent = process_batches(test_dataloader_title)

##RoBERTa
# Body
train_dataloader_body = create_dataloader(traindf["body"].values.tolist(), batch_size)
test_dataloader_body = create_dataloader(testdf["body"].values.tolist(), batch_size)
cls_train_body_rob = process_batches_roberta(train_dataloader_body)
cls_test_body_rob = process_batches_roberta(test_dataloader_body)

#Title
train_dataloader_title = create_dataloader(traindf["title"].values.tolist(), batch_size)
test_dataloader_title = create_dataloader(testdf["title"].values.tolist(), batch_size)
cls_train_title_rob = process_batches_roberta(train_dataloader_title)
cls_test_title_rob = process_batches_roberta(test_dataloader_title)

# Features Processing

In [21]:
emb_array_train  = []
emb_array_test = []

emb_sent_sent_train = np.concatenate((cls_train_body_sent.detach().cpu().numpy(), cls_train_title_sent.detach().cpu().numpy()), axis=1)
emb_sent_sent_test = np.concatenate((cls_test_body_sent.detach().cpu().numpy(), cls_test_title_sent.detach().cpu().numpy()), axis=1)

emb_sent_rob_train = np.concatenate((cls_train_body_sent.detach().cpu().numpy(), cls_train_title_rob.detach().cpu().numpy()), axis=1)
emb_sent_rob_test = np.concatenate((cls_test_body_sent.detach().cpu().numpy(), cls_test_title_rob.detach().cpu().numpy()), axis=1)

emb_rob_sent_train = np.concatenate((cls_train_body_rob.detach().cpu().numpy(), cls_train_title_sent.detach().cpu().numpy()), axis=1)
emb_rob_sent_test = np.concatenate((cls_test_body_rob.detach().cpu().numpy(), cls_test_title_sent.detach().cpu().numpy()), axis=1)

emb_rob_rob_train = np.concatenate((cls_train_body_rob.detach().cpu().numpy(), cls_train_title_rob.detach().cpu().numpy()), axis=1)
emb_rob_rob_test = np.concatenate((cls_test_body_rob.detach().cpu().numpy(), cls_test_title_rob.detach().cpu().numpy()), axis=1)

scaler = MinMaxScaler()

emb_array_train.append(scaler.fit_transform(emb_sent_sent_train))
emb_array_test.append(scaler.transform(emb_sent_sent_test))

emb_array_train.append(scaler.fit_transform(emb_sent_rob_train))
emb_array_test.append(scaler.transform(emb_sent_rob_test))

emb_array_train.append(scaler.fit_transform(emb_rob_sent_train))
emb_array_test.append(scaler.transform(emb_rob_sent_test))

emb_array_train.append(scaler.fit_transform(emb_rob_rob_train))
emb_array_test.append(scaler.transform(emb_rob_rob_test))

emb_array_train = np.array(emb_array_train)
emb_array_test = np.array(emb_array_test)

In [22]:
x_train = emb_array_train
y_train = np.array([traindf["label"], traindf["label"], traindf["label"], traindf["label"]])
x_test = emb_array_test
y_test = np.array([testdf["label"], testdf["label"], testdf["label"], testdf["label"]])

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(4, 1500, 1536) (4, 1500)
(4, 1498, 1536) (4, 1498)


In [23]:
import pickle

with open("x_train.pkl", "wb") as f:
    pickle.dump(x_train, f)
with open("y_train.pkl", "wb") as f:
    pickle.dump(y_train, f)
with open("x_test.pkl", "wb") as f:
    pickle.dump(x_test, f)
with open("y_test.pkl", "wb") as f:
    pickle.dump(y_test, f)

#Models

## LSTM

In [24]:
import pickle
with open("x_train.pkl", "rb") as f:
      x_train = pickle.load(f)
with open("y_train.pkl", "rb") as f:
    y_train = pickle.load(f)
with open("x_test.pkl", "rb") as f:
    x_test = pickle.load(f)
with open("y_test.pkl", "rb") as f:
    y_test = pickle.load(f)

In [26]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

In [27]:
import torch
import torch.nn as nn

# LSTM Classifier with fixed hidden dimensions (256 for first layer, 128 for second layer)
def reset_lstm(input_dim, num_classes, dropout, device):
    class LSTMClassifier(nn.Module):
        def __init__(self, input_dim, num_classes, dropout):
            super(LSTMClassifier, self).__init__()

            # Define LSTM layers with fixed hidden dimensions (256 for first layer, 128 for second layer)
            self.lstm1 = nn.LSTM(input_dim, 64, num_layers=2, batch_first=True, dropout=dropout)

            # Fully connected layer
            self.fc = nn.Linear(64, num_classes)  # Using 128 as the input to the fc layer

        def forward(self, x):
            # x: [batch_size, seq_len, input_dim]

            # Pass input through the first LSTM layer
            out1, (hn1, _) = self.lstm1(x)  # out1: [batch_size, seq_len, 256], hn1: [2, batch_size, 256]

            # Get the last hidden state of the second LSTM layer (hn2 is of shape [2, batch_size, 128])
            x = hn1[-1]  # Shape: [batch_size, 128]

            # Pass through the fully connected layer
            x = self.fc(x)  # Output: [batch_size, num_classes]
            return x

    # Create and return the model, moving it to the specified device (GPU/CPU)
    model = LSTMClassifier(input_dim, num_classes, dropout)
    return model.to(device)

In [28]:
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return {
            "embedding": torch.tensor(self.embeddings[idx], dtype=torch.float),
            "label": torch.tensor(self.labels[idx], dtype=torch.long),
        }

In [29]:
for i in range(4):
    # Use all of x_train and y_train for training
    train_embeddings = x_train[i]
    train_labels = y_train[i]

    # Use x_test and y_test for evaluation
    test_embeddings = x_test[i]
    test_labels = y_test[i]

    # Create datasets
    train_dataset = EmbeddingDataset(train_embeddings, train_labels)
    test_dataset = EmbeddingDataset(test_embeddings, test_labels)

    # DataLoaders for training and test datasets
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Initialize and train the final model
    final_model = reset_lstm(
        input_dim=1536,
        num_classes=3,
        dropout=0.2,
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

    optimizer = torch.optim.Adam(final_model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(100):
        final_model.train()
        for batch in train_loader:
            embeddings = batch["embedding"].unsqueeze(1).to(device)  # [batch_size, 1, input_dim]
            labels = batch["label"].to(device)  # [batch_size]

            optimizer.zero_grad()
            outputs = final_model(embeddings)  # [batch_size, num_classes]

            # Ensure the output shape is [batch_size, num_classes]
            loss = criterion(outputs.view(embeddings.shape[0], 3), labels)  # CrossEntropyLoss expects outputs of shape [batch_size, num_classes] and labels of shape [batch_size]
            loss.backward()
            optimizer.step()

    # Evaluate on test set
    final_model.eval()
    test_preds, test_labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            embeddings = batch["embedding"].unsqueeze(1).to(device)  # [batch_size, 1, input_dim]
            labels = batch["label"].to(device)  # [batch_size]
            outputs = final_model(embeddings)
            preds = torch.argmax(outputs, dim=1)
            test_preds.extend(preds.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(test_labels, test_preds)
    f1 = f1_score(test_labels, test_preds, average="weighted")
    precision = precision_score(test_labels, test_preds, average="weighted")
    recall = recall_score(test_labels, test_preds, average="weighted")

    print(f"Combination {i+1} Results:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")


Combination 1 Results:
Accuracy: 0.7243
F1 Score: 0.7129
Precision: 0.7382
Recall: 0.7243
Combination 2 Results:
Accuracy: 0.7437
F1 Score: 0.7431
Precision: 0.7432
Recall: 0.7437
Combination 3 Results:
Accuracy: 0.7417
F1 Score: 0.7332
Precision: 0.7607
Recall: 0.7417
Combination 4 Results:
Accuracy: 0.7377
F1 Score: 0.7377
Precision: 0.7389
Recall: 0.7377


## DistilBERT + Adapters

In [30]:
import pickle
with open("x_train.pkl", "rb") as f:
    x_train = pickle.load(f)
with open("y_train.pkl", "rb") as f:
    y_train = pickle.load(f)
with open("x_test.pkl", "rb") as f:
    x_test = pickle.load(f)
with open("y_test.pkl", "rb") as f:
    y_test = pickle.load(f)

In [31]:
class EmbeddingDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return {
            "embedding": torch.tensor(self.embeddings[idx], dtype=torch.float),
            "label": torch.tensor(self.labels[idx], dtype=torch.long),
        }

In [32]:
for i in range(len(x_train)):
  print(f"Training model for combination {i+1}")

  # Prepare datasets and dataloaders
  train_dataset = EmbeddingDataset(x_train[i], y_train[i])
  test_dataset = EmbeddingDataset(x_test[i], y_test[i])

  train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

  # Load DistilBert with Adapters
  config = DistilBertConfig.from_pretrained("distilbert-base-uncased")
  model = DistilBertAdapterModel.from_pretrained("distilbert-base-uncased", config=config)

  # Add adapter
  adapter_config = AdapterConfig.load("pfeiffer")
  model.add_adapter("classification_adapter", config=adapter_config)
  model.train_adapter("classification_adapter")

  # Add classification head
  model.add_classification_head(
      "classification_adapter", num_labels=3, id2label={0: "class_0", 1: "class_1", 2: "class_2"}
  )

  optimizer = torch.optim.AdamW(
    list(model.parameters()),
    lr=1e-3)

  criterion = torch.nn.CrossEntropyLoss()

  # Training loop
  model.train()
  # projection.train()  # Ensure projection layer is in training mode
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  for epoch in range(200):  # Fixed number of epochs for simplicity
      total_loss = 0
      for batch in train_loader:
          embeddings = batch["embedding"].view(batch["embedding"].shape[0], 2, 768).to(device)  # Shape: [batch_size, hidden_dim]
          labels = batch["label"].to(device)         # Shape: [batch_size]

          optimizer.zero_grad()
          outputs = model(inputs_embeds=embeddings, adapter_name="classification_adapter")
          logits = outputs.logits.squeeze(1)  # Remove the sequence length dimension
          loss = criterion(logits, labels)
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

  # Evaluation
  model.eval()
  preds, true_labels = [], []

  with torch.no_grad():
      for batch in test_loader:
          embeddings = batch["embedding"].view(batch["embedding"].shape[0], 2, 768).to(device)  # Shape: [batch_size, hidden_dim]
          labels = batch["label"].to(device)         # Shape: [batch_size]

          # Add sequence length dimension and forward pass
          outputs = model(inputs_embeds=embeddings, adapter_name="classification_adapter")
          logits = outputs.logits.squeeze(1)  # Shape: [batch_size, num_labels]

          # Predictions
          predictions = torch.argmax(logits, dim=1)  # Shape: [batch_size]

          # # Debugging (ensure shapes match)
          # print(f"Batch size (labels): {labels.size()}, Batch size (predictions): {predictions.size()}")

          # Accumulate predictions and true labels
          preds.extend(predictions.cpu().numpy())
          true_labels.extend(labels.cpu().numpy())

  # # Check for mismatches before computing metrics
  # print(f"Total samples in preds: {len(preds)}, Total samples in true_labels: {len(true_labels)}")

  if len(preds) == len(true_labels):
      accuracy = accuracy_score(true_labels, preds)
      f1 = f1_score(true_labels, preds, average="weighted")
      precision = precision_score(true_labels, preds, average="weighted")
      recall = recall_score(true_labels, preds, average="weighted")
      print(f"\n-----------------------------------------------------------")
      print(f"Accuracy: {accuracy:.4f}")
      print(f"F1 Score: {f1:.4f}")
      print(f"Precision: {precision:.4f}")
      print(f"Recall: {recall:.4f}")
      print(f"-----------------------------------------------------------")
  else:
      print("Mismatch in number of predictions and true labels!")


Training model for combination 1


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertAdapterModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


Epoch 1, Loss: 0.9843
Epoch 2, Loss: 0.7617
Epoch 3, Loss: 0.6623
Epoch 4, Loss: 0.5629
Epoch 5, Loss: 0.4520
Epoch 6, Loss: 0.3634
Epoch 7, Loss: 0.3301
Epoch 8, Loss: 0.2644
Epoch 9, Loss: 0.2432
Epoch 10, Loss: 0.2149
Epoch 11, Loss: 0.2043
Epoch 12, Loss: 0.1906
Epoch 13, Loss: 0.1405
Epoch 14, Loss: 0.1441
Epoch 15, Loss: 0.1062
Epoch 16, Loss: 0.1001
Epoch 17, Loss: 0.1151
Epoch 18, Loss: 0.0984
Epoch 19, Loss: 0.0746
Epoch 20, Loss: 0.0904
Epoch 21, Loss: 0.0684
Epoch 22, Loss: 0.0738
Epoch 23, Loss: 0.1011
Epoch 24, Loss: 0.0569
Epoch 25, Loss: 0.0744
Epoch 26, Loss: 0.0812
Epoch 27, Loss: 0.0658
Epoch 28, Loss: 0.0632
Epoch 29, Loss: 0.0382
Epoch 30, Loss: 0.0360
Epoch 31, Loss: 0.0571
Epoch 32, Loss: 0.0511
Epoch 33, Loss: 0.0492
Epoch 34, Loss: 0.0316
Epoch 35, Loss: 0.0699
Epoch 36, Loss: 0.0649
Epoch 37, Loss: 0.0375
Epoch 38, Loss: 0.0564
Epoch 39, Loss: 0.0449
Epoch 40, Loss: 0.0524
Epoch 41, Loss: 0.0356
Epoch 42, Loss: 0.0425
Epoch 43, Loss: 0.0443
Epoch 44, Loss: 0.04

In [33]:
for i in range(len(x_train)):
  print(f"Training model for combination {i+1}")

  # Prepare datasets and dataloaders
  train_dataset = EmbeddingDataset(x_train[i], y_train[i])
  test_dataset = EmbeddingDataset(x_test[i], y_test[i])

  train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

  # Load Roberta with Adapters
  config = RobertaConfig.from_pretrained("roberta-base")
  model = RobertaAdapterModel.from_pretrained("roberta-base", config=config)

  # Add adapter
  adapter_config = AdapterConfig.load("pfeiffer")
  model.add_adapter("classification_adapter", config=adapter_config)
  model.train_adapter("classification_adapter")

  # Add classification head
  model.add_classification_head(
      "classification_adapter", num_labels=3, id2label={0: "class_0", 1: "class_1", 2: "class_2"}
  )

  optimizer = torch.optim.AdamW(
    list(model.parameters()),
    lr=1e-3)

  criterion = torch.nn.CrossEntropyLoss()

  # Training loop
  model.train()
  # projection.train()  # Ensure projection layer is in training mode
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  for epoch in range(100):  # Fixed number of epochs for simplicity
      total_loss = 0
      for batch in train_loader:
          embeddings = batch["embedding"].view(batch["embedding"].shape[0], 2, 768).to(device)  # Shape: [batch_size, hidden_dim]
          labels = batch["label"].to(device)         # Shape: [batch_size]

          optimizer.zero_grad()
          outputs = model(inputs_embeds=embeddings, adapter_name="classification_adapter")
          logits = outputs.logits.squeeze(1)  # Remove the sequence length dimension
          loss = criterion(logits, labels)
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

  # Evaluation
  model.eval()
  preds, true_labels = [], []

  with torch.no_grad():
      for batch in test_loader:
          embeddings = batch["embedding"].view(batch["embedding"].shape[0], 2, 768).to(device)  # Shape: [batch_size, hidden_dim]
          labels = batch["label"].to(device)         # Shape: [batch_size]

          # Add sequence length dimension and forward pass
          outputs = model(inputs_embeds=embeddings, adapter_name="classification_adapter")
          logits = outputs.logits.squeeze(1)  # Shape: [batch_size, num_labels]

          # Predictions
          predictions = torch.argmax(logits, dim=1)  # Shape: [batch_size]

          # # Debugging (ensure shapes match)
          # print(f"Batch size (labels): {labels.size()}, Batch size (predictions): {predictions.size()}")

          # Accumulate predictions and true labels
          preds.extend(predictions.cpu().numpy())
          true_labels.extend(labels.cpu().numpy())

  # # Check for mismatches before computing metrics
  # print(f"Total samples in preds: {len(preds)}, Total samples in true_labels: {len(true_labels)}")

  if len(preds) == len(true_labels):
      accuracy = accuracy_score(true_labels, preds)
      f1 = f1_score(true_labels, preds, average="weighted")
      precision = precision_score(true_labels, preds, average="weighted")
      recall = recall_score(true_labels, preds, average="weighted")
      print(f"\n-----------------------------------------------------------")
      print(f"Accuracy: {accuracy:.4f}")
      print(f"F1 Score: {f1:.4f}")
      print(f"Precision: {precision:.4f}")
      print(f"Recall: {recall:.4f}")
      print(f"-----------------------------------------------------------")
  else:
      print("Mismatch in number of predictions and true labels!")


Training model for combination 1


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

RobertaAdapterModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.

Epoch 1, Loss: 1.1342
Epoch 2, Loss: 1.1136
Epoch 3, Loss: 1.0981
Epoch 4, Loss: 1.0752
Epoch 5, Loss: 1.0797
Epoch 6, Loss: 1.0987
Epoch 7, Loss: 1.1022
Epoch 8, Loss: 1.1045
Epoch 9, Loss: 1.1018
Epoch 10, Loss: 1.1074
Epoch 11, Loss: 1.1016
Epoch 12, Loss: 1.0210
Epoch 13, Loss: 1.0344
Epoch 14, Loss: 1.0143
Epoch 15, Loss: 1.0031
Epoch 16, Loss: 1.0421
Epoch 17, Loss: 0.9777
Epoch 18, Loss: 0.9683
Epoch 19, Loss: 0.9664
Epoch 20, Loss: 0.9522
Epoch 21, Loss: 0.9349
Epoch 22, Loss: 0.9278
Epoch 23, Loss: 0.9091
Epoch 24, Loss: 0.8918
Epoch 25, Loss: 0.9287
Epoch 26, Loss: 0.9211
Epoch 27, Loss: 0.9049
Epoch 28, Loss: 0.9197
Epoch 29, Loss: 0.9616
Epoch 30, Loss: 0.9107
Epoch 31, Loss: 0.9105
Epoch 32, Loss: 0.8827
Epoch 33, Loss: 0.8602
Epoch 34, Loss: 0.8563
Epoch 35, Loss: 0.9124
Epoch 36, Loss: 0.8474
Epoch 37, Loss: 0.8399
Epoch 38, Loss: 0.8201
Epoch 39, Loss: 0.8192
Epoch 40, Loss: 0.8236
Epoch 41, Loss: 0.8283
Epoch 42, Loss: 0.8111
Epoch 43, Loss: 0.7611
Epoch 44, Loss: 0.74

...

Epoch 1, Loss: 1.1230
Epoch 2, Loss: 1.1177
Epoch 3, Loss: 1.1164
Epoch 4, Loss: 1.1080
Epoch 5, Loss: 1.1087
Epoch 6, Loss: 1.1068
Epoch 7, Loss: 1.1079
Epoch 8, Loss: 1.1057
Epoch 9, Loss: 1.1051
Epoch 10, Loss: 1.1052
Epoch 11, Loss: 1.1031
Epoch 12, Loss: 1.1009
Epoch 13, Loss: 1.1056
Epoch 14, Loss: 1.1001
Epoch 15, Loss: 1.1037
Epoch 16, Loss: 1.1035
Epoch 17, Loss: 1.0906
Epoch 18, Loss: 1.1053
Epoch 19, Loss: 1.0548
Epoch 20, Loss: 1.0227
Epoch 21, Loss: 1.0247
Epoch 22, Loss: 1.0403
Epoch 23, Loss: 1.0524
Epoch 24, Loss: 1.0586
Epoch 25, Loss: 1.0846
Epoch 26, Loss: 1.0833
Epoch 27, Loss: 1.0902
Epoch 28, Loss: 1.0796
Epoch 29, Loss: 1.0718
Epoch 30, Loss: 1.0797
Epoch 31, Loss: 1.0923
Epoch 32, Loss: 1.0590
Epoch 33, Loss: 1.1183
Epoch 34, Loss: 1.1138
Epoch 35, Loss: 1.1009
Epoch 36, Loss: 1.1030
Epoch 37, Loss: 1.1024
Epoch 38, Loss: 1.1019
Epoch 39, Loss: 1.1017
Epoch 40, Loss: 1.1015
Epoch 41, Loss: 1.1014
Epoch 42, Loss: 1.1016
Epoch 43, Loss: 1.1008
Epoch 44, Loss: 1.10

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



-----------------------------------------------------------
Accuracy: 0.3338
F1 Score: 0.1671
Precision: 0.1114
Recall: 0.3338
-----------------------------------------------------------
Training model for combination 4


Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 1.1272
Epoch 2, Loss: 1.1145
Epoch 3, Loss: 1.1109
Epoch 4, Loss: 1.1078
Epoch 5, Loss: 1.1155
Epoch 6, Loss: 1.1045
Epoch 7, Loss: 1.1046
Epoch 8, Loss: 1.1023
Epoch 9, Loss: 1.0802
Epoch 10, Loss: 1.0513
Epoch 11, Loss: 1.0071
Epoch 12, Loss: 1.0069
Epoch 13, Loss: 1.0540
Epoch 14, Loss: 1.0264
Epoch 15, Loss: 0.9575
Epoch 16, Loss: 0.9587
Epoch 17, Loss: 0.9382
Epoch 18, Loss: 0.9060
Epoch 19, Loss: 0.8977
Epoch 20, Loss: 0.8635
Epoch 21, Loss: 0.8459
Epoch 22, Loss: 0.8438
Epoch 23, Loss: 0.8220
Epoch 24, Loss: 0.8057
Epoch 25, Loss: 0.7789
Epoch 26, Loss: 0.8013
Epoch 27, Loss: 0.7701
Epoch 28, Loss: 0.7227
Epoch 29, Loss: 0.7239
Epoch 30, Loss: 0.7078
Epoch 31, Loss: 0.7548
Epoch 32, Loss: 0.7030
Epoch 33, Loss: 0.7037
Epoch 34, Loss: 0.6728
Epoch 35, Loss: 0.6223
Epoch 36, Loss: 0.6036
Epoch 37, Loss: 0.5921
Epoch 38, Loss: 0.6013
Epoch 39, Loss: 0.5589
Epoch 40, Loss: 0.5536
Epoch 41, Loss: 0.5566
Epoch 42, Loss: 0.5270
Epoch 43, Loss: 0.5214
Epoch 44, Loss: 0.57

## SVM

In [35]:
for i in range(len(x_train)):
  svc = SVC(C=1.6)
  svc.fit(x_train[i], y_train[i])
  y_preds = svc.predict(x_test[i])

  print(accuracy_score(y_test[i], y_preds))
  print(f1_score(y_test[i], y_preds, average='micro'))
  print(precision_score(y_test[i], y_preds, average='micro'))
  print(recall_score(y_test[i], y_preds, average='micro'))

0.7676902536715621
0.7676902536715621
0.7676902536715621
0.7676902536715621
0.753004005340454
0.753004005340454
0.753004005340454
0.753004005340454
0.7857142857142857
0.7857142857142857
0.7857142857142857
0.7857142857142857
0.7556742323097463
0.7556742323097463
0.7556742323097463
0.7556742323097463


In [37]:
for i in range(len(x_train)):
  param_grid = {
      'C': [1.5, 1.6, 1.7],
      'kernel': ['linear', 'rbf', 'poly'],
      'gamma': [0.1, 1],
      'degree': [2, 3]
  }

  svc = SVC()
  grid_search_svc = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy', verbose=2)

  grid_search_svc.fit(x_train[i], y_train[i])

  print("Best Parameters for SVC:", grid_search_svc.best_params_)
  print("Best Accuracy for SVC:", grid_search_svc.best_score_)

  best_model = grid_search_svc.best_estimator_
  y_preds = best_model.predict(x_test[i])

  print(accuracy_score(y_test[i], y_preds))
  print(f1_score(y_test[i], y_preds, average='micro'))
  print(precision_score(y_test[i], y_preds, average='micro'))
  print(recall_score(y_test[i], y_preds, average='micro'))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END ..........C=1.5, degree=2, gamma=0.1, kernel=linear; total time=   1.0s
[CV] END ..........C=1.5, degree=2, gamma=0.1, kernel=linear; total time=   1.2s
[CV] END ..........C=1.5, degree=2, gamma=0.1, kernel=linear; total time=   1.2s
[CV] END ..........C=1.5, degree=2, gamma=0.1, kernel=linear; total time=   1.2s
[CV] END ..........C=1.5, degree=2, gamma=0.1, kernel=linear; total time=   1.4s
[CV] END .............C=1.5, degree=2, gamma=0.1, kernel=rbf; total time=   2.9s
[CV] END .............C=1.5, degree=2, gamma=0.1, kernel=rbf; total time=   1.9s
[CV] END .............C=1.5, degree=2, gamma=0.1, kernel=rbf; total time=   2.0s
[CV] END .............C=1.5, degree=2, gamma=0.1, kernel=rbf; total time=   1.8s
[CV] END .............C=1.5, degree=2, gamma=0.1, kernel=rbf; total time=   1.8s
[CV] END ............C=1.5, degree=2, gamma=0.1, kernel=poly; total time=   3.1s
[CV] END ............C=1.5, degree=2, gamma=0.1

## Random Forest

In [38]:
for i in range(len(x_train)):
  param_grid_rf = {
      'n_estimators': [100, 200, 300],
      'max_depth': [2, 4, 5],
      'min_samples_split': [2, 5],
      'min_samples_leaf': [1, 2],
  }

  rf = RandomForestClassifier(random_state=42)
  grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', verbose=2)


  grid_search_rf.fit(x_train[i], y_train[i])

  print("Best Parameters for Random Forest:", grid_search_rf.best_params_)
  print("Best Accuracy for Random Forest:", grid_search_rf.best_score_)

  best_model = grid_search_rf.best_estimator_
  y_preds = best_model.predict(x_test[i])

  print(accuracy_score(y_test[i], y_preds))
  print(f1_score(y_test[i], y_preds, average='micro'))
  print(precision_score(y_test[i], y_preds, average='micro'))
  print(recall_score(y_test[i], y_preds, average='micro'))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.0s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.2s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.9s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.1s
[CV] END max_depth=2, min_samples_leaf

## XGBoost

In [ ]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# # Define the parameter grid for XGBClassifier
# param_grid = {
#     'n_estimators': [50, 100],          # Number of trees
#     'max_depth': [2, 3],              # Maximum depth of each tree
#     'learning_rate': [0.1, 0.01],        # Step size shrinkage
#     'subsample': [0.8, 1.0],             # Subsample ratio of the training instances
#     'colsample_bytree': [0.8, 1.0]       # Subsample ratio of columns when constructing each tree
# }

# # Initialize an XGBClassifier
# xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=7)

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=xgb_model,
#                            param_grid=param_grid,
#                            scoring='f1_micro',   # You can use 'accuracy', 'f1_macro', etc. as needed
#                            cv=3,                 # 3-fold cross-validation
#                            verbose=1,            # Verbosity to see progress
#                            n_jobs=-1)            # Use all available cores

# # Fit the model to the training data
# grid_search.fit(x_train, y_train)

# # Extract the best model from the grid search
# best_model = grid_search.best_estimator_

# # Print the best parameters
# print("Best Parameters found by Grid Search:", grid_search.best_params_)

# # Predict on the test set
# y_pred = best_model.predict(x_test)

# # Calculate metrics
# accuracy = accuracy_score(y_test, y_pred)
# precision_micro = precision_score(y_test, y_pred, average='micro')
# recall_micro = recall_score(y_test, y_pred, average='micro')
# f1_micro = f1_score(y_test, y_pred, average='micro')

# print("Accuracy:", accuracy)
# print("Precision:", precision_micro)
# print("Recall:", recall_micro)
# print("F1 Score:", f1_micro)

# # Optional: detailed classification report
# print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [39]:
for i in range(len(x_train)):

    # Initialize the XGBClassifier with the best parameters
    best_xgb_model = XGBClassifier(
        colsample_bytree=1.0,
        learning_rate=0.1,
        max_depth=2,
        n_estimators=100,
        subsample=1.0,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=7
    )

    # Fit the model to the training data
    best_xgb_model.fit(x_train[i], [y_train[i]])

    # Predict on the test set
    y_pred = best_xgb_model.predict(x_test[i])

    # Calculate metrics
    accuracy = accuracy_score(y_test[i], y_pred)
    precision_micro = precision_score(y_test[i], y_pred, average='micro')
    recall_micro = recall_score(y_test[i], y_pred, average='micro')
    f1_micro = f1_score(y_test[i], y_pred, average='micro')

    # Print metrics
    print("Final Model Performance with Best Parameters:")
    print("Accuracy:", accuracy)
    print("Precision (micro):", precision_micro)
    print("Recall (micro):", recall_micro)
    print("F1 Score (micro):", f1_micro)

    # Optional: detailed classification report
    print("\nClassification Report:\n", classification_report(y_test[i], y_pred))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:42:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Model Performance with Best Parameters:
Accuracy: 0.7162883845126836
Precision (micro): 0.7162883845126836
Recall (micro): 0.7162883845126836
F1 Score (micro): 0.7162883845126836

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.70      0.72       500
           1       0.72      0.83      0.78       500
           2       0.67      0.62      0.64       498

    accuracy                           0.72      1498
   macro avg       0.72      0.72      0.71      1498
weighted avg       0.72      0.72      0.71      1498



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:42:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Model Performance with Best Parameters:
Accuracy: 0.7142857142857143
Precision (micro): 0.7142857142857143
Recall (micro): 0.7142857142857143
F1 Score (micro): 0.7142857142857143

Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.69      0.71       500
           1       0.73      0.82      0.77       500
           2       0.69      0.63      0.66       498

    accuracy                           0.71      1498
   macro avg       0.71      0.71      0.71      1498
weighted avg       0.71      0.71      0.71      1498



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:43:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Model Performance with Best Parameters:
Accuracy: 0.7323097463284379
Precision (micro): 0.7323097463284379
Recall (micro): 0.7323097463284379
F1 Score (micro): 0.7323097463284379

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.77      0.76       500
           1       0.74      0.79      0.76       500
           2       0.71      0.63      0.67       498

    accuracy                           0.73      1498
   macro avg       0.73      0.73      0.73      1498
weighted avg       0.73      0.73      0.73      1498



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:43:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Final Model Performance with Best Parameters:
Accuracy: 0.7209612817089452
Precision (micro): 0.7209612817089452
Recall (micro): 0.7209612817089452
F1 Score (micro): 0.7209612817089452

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.74      0.74       500
           1       0.72      0.79      0.75       500
           2       0.71      0.64      0.67       498

    accuracy                           0.72      1498
   macro avg       0.72      0.72      0.72      1498
weighted avg       0.72      0.72      0.72      1498



## LightGBM

In [ ]:
# from lightgbm import LGBMClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# # Define an expanded parameter grid for LGBMClassifier
# param_grid = {
#     'n_estimators': [50, 100],         # Number of boosting iterations
#     'max_depth': [3, 5],                 # Tree depth
#     'learning_rate': [0.1, 0.05],           # Different learning rates
#     'num_leaves': [15, 31],             # Number of leaves
#     'subsample': [0.7, 0.8],           # Subsampling ratio for rows
#     'colsample_bytree': [0.7, 0.8],    # Subsampling ratio for features
#     'min_child_samples': [10, 20],      # Minimum number of samples per leaf
#     'reg_alpha': [0.0, 0.1],           # L1 regularization strength
#     'reg_lambda': [0.0, 0.1]           # L2 regularization strength
# }

# # Initialize an LGBMClassifier
# lgb_model = LGBMClassifier(random_state=7)

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=lgb_model,
#                            param_grid=param_grid,
#                            scoring='f1_micro',  # Use 'accuracy' or 'f1_macro' if preferred
#                            cv=3,                # 3-fold cross-validation
#                            verbose=1,           # Show progress
#                            n_jobs=-1)           # Use all available cores

# # Fit the model to the training data
# grid_search.fit(x_train, y_train)

# # Extract the best model from the grid search
# best_model = grid_search.best_estimator_

# # Print the best parameters
# print("Best Parameters found by Grid Search:", grid_search.best_params_)

# # Predict on the test set
# y_pred = best_model.predict(x_test)

# # Calculate metrics
# accuracy = accuracy_score(y_test, y_pred)
# precision_micro = precision_score(y_test, y_pred, average='micro')
# recall_micro = recall_score(y_test, y_pred, average='micro')
# f1_micro = f1_score(y_test, y_pred, average='micro')

# print("Accuracy:", accuracy)
# print("Precision:", precision_micro)
# print("Recall:", recall_micro)
# print("F1 Score:", f1_micro)

# # Optional: detailed classification report
# print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [41]:
for i in range(len(x_train)):
    # Initialize the LGBMClassifier with the best parameters
    best_lgb_model = LGBMClassifier(
        colsample_bytree=0.7,
        learning_rate=0.1,
        max_depth=5,
        min_child_samples=20,
        n_estimators=100,
        num_leaves=15,
        reg_alpha=0.1,
        reg_lambda=0.0,
        subsample=0.7,
        random_state=7
    )

    # Fit the model to the training data
    best_lgb_model.fit(x_train[i], y_train[i])

    # Predict on the test set
    y_pred = best_lgb_model.predict(x_test[i])

    # Calculate metrics
    accuracy = accuracy_score(y_test[i], y_pred)
    precision_micro = precision_score(y_test[i], y_pred, average='micro')
    recall_micro = recall_score(y_test[i], y_pred, average='micro')
    f1_micro = f1_score(y_test[i], y_pred, average='micro')

    # Print metrics
    print("Final Model Performance with Best Parameters:")
    print("Accuracy:", accuracy)
    print("Precision (micro):", precision_micro)
    print("Recall (micro):", recall_micro)
    print("F1 Score (micro):", f1_micro)

    # Optional: detailed classification report
    print("\nClassification Report:\n", classification_report(y_test[i], y_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.049891 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 391680
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 1536
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Final Model Performance with Best Parameters:
Accuracy: 0.739652870493992
Precision (micro): 0.739652870493992
Recall (micro): 0.739652870493992
F1 Score (micro): 0.739652870493992

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.72      0.75       500
           1       0.75      0.85      0.80       500
           2       0.69      0.64      0.66       498

    accuracy                           0.74      1498
   macro avg       0.74      0.74      0.74      1498
weighted avg       0.74      0.74      0.74      1498

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031396 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 391680
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 1536
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [I

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Final Model Performance with Best Parameters:
Accuracy: 0.7423230974632844
Precision (micro): 0.7423230974632844
Recall (micro): 0.7423230974632844
F1 Score (micro): 0.7423230974632844

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.72      0.73       500
           1       0.76      0.82      0.79       500
           2       0.71      0.69      0.70       498

    accuracy                           0.74      1498
   macro avg       0.74      0.74      0.74      1498
weighted avg       0.74      0.74      0.74      1498

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043319 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 391680
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 1536
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Final Model Performance with Best Parameters:
Accuracy: 0.7550066755674232
Precision (micro): 0.7550066755674232
Recall (micro): 0.7550066755674232
F1 Score (micro): 0.7550066755674232

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.75      0.77       500
           1       0.75      0.83      0.79       500
           2       0.72      0.69      0.71       498

    accuracy                           0.76      1498
   macro avg       0.76      0.75      0.75      1498
weighted avg       0.76      0.76      0.75      1498

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.028546 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 391680
[LightGBM] [Info] Number of data points in the train set: 1500, number of used features: 1536
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
